<a href="https://colab.research.google.com/github/gamecicn/TF2_Study/blob/KaggleCrowdFlowerSearch/T2E1_Kaggle_MINST_CNN3.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf

In [ ]:
# Download Data
csv_train_file = '/content/drive/MyDrive/DS_data/MINSTHW/train.csv'
csv_test_file = '/content/drive/MyDrive/DS_data/MINSTHW/test.csv'

df_data = pd.read_csv(csv_train_file)
df_test = pd.read_csv(csv_test_file)

df_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Preprcessing

In [ ]:
# Split train data to train and valid dataset
df_data_copy = df_data.copy()

df_train  = df_data_copy.sample(frac=0.99, random_state=0)
df_valid = df_data_copy.drop(df_train.index)

print("lenght of train is {}".format(len(df_train)))
print("lenght of valid is {}".format(len(df_valid)))

train_labels = df_train.pop('label')
valid_labels = df_valid.pop('label')

df_train.head()

#np.apply_along_axis(lambda x : np.reshape, 1, df_train.to_numpy(), 28, 28)
df_train = df_train.to_numpy().reshape(-1, 28 ,28, 1)
df_valid = df_valid.to_numpy().reshape(-1, 28 ,28, 1)

lenght of train is 41580
lenght of valid is 420


## Build the model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(28, (2, 2), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(48, (2, 2), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(48, (2, 2), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 27, 27, 28)        140       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 13, 13, 28)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 13, 13, 28)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 48)        5424      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 6, 6, 48)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 6, 6, 48)          0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 5, 5, 48)        

### Training

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(df_train, 
                    train_labels,
                    batch_size=64,
                    epochs=50)

Epoch 1/50
650/650 [==============================] - 2s 3ms/step - loss: 0.3447 - accuracy: 0.8922
Epoch 2/50
650/650 [==============================] - 2s 3ms/step - loss: 0.2147 - accuracy: 0.9315
Epoch 3/50
650/650 [==============================] - 2s 3ms/step - loss: 0.1565 - accuracy: 0.9502
Epoch 4/50
650/650 [==============================] - 2s 3ms/step - loss: 0.1332 - accuracy: 0.9568
Epoch 5/50
650/650 [==============================] - 2s 3ms/step - loss: 0.1189 - accuracy: 0.9613
Epoch 6/50
650/650 [==============================] - 2s 3ms/step - loss: 0.1073 - accuracy: 0.9645
Epoch 7/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0940 - accuracy: 0.9692
Epoch 8/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0894 - accuracy: 0.9719
Epoch 9/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0892 - accuracy: 0.9716
Epoch 10/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0805 - accuracy: 0.9734

### Validate

In [ ]:
 val_loss, val_acc = model.evaluate(df_valid,  valid_labels, verbose=2)

14/14 - 0s - loss: 0.0228 - accuracy: 0.9976


## Make predictions

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(df_test.to_numpy().reshape(-1, 28, 28, 1))

res = pd.DataFrame(data = {"ImageId" : np.arange(1, 1+len(predictions)) , 
                           "Label"   : list(map(np.argmax, predictions))})

In [ ]:


res.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [ ]:
res.to_csv("/content/drive/MyDrive/output/Kaggle_minst_hw_cnn4.csv", index=False)